In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from plaid.esmfold import esmfold_v1
from plaid.decoder import FullyConnectedNetwork

mlp = FullyConnectedNetwork.from_pretrained()
# path = "../cached_tensors/decoder_vocab_21.ckpt"
# path = "/shared/amyxlu/plaid/checkpoints/sequence_decoder/2024-01-19T02-19-28/last.ckpt"
# mlp = FullyConnectedNetwork.load_from_checkpoint(path, n_classes=21, mlp_num_layers=3, esmfold=None)

In [3]:
device = torch.device("cuda:0") 
esmfold = esmfold_v1()
esmfold = esmfold.to(device).eval().requires_grad_(False)

Creating ESMFold...
ESMFold model loaded in 45.42 seconds.


In [75]:
s = """MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVISEPKPEKKEDIQLK\
KKEVVAVAKKEEVLKKEVVVPSKKDEEILPLKKEVPRPPKKEEDVMPQKKEVPRPPKKEE\
DIVPQMRDVSLPPKEEEKIVPKKKEVPRPPKKVEEILPPKKEVHRPPKKEEDIVPQIREV\
SLPPKKDEEIVCEKKEVAPAKEEPSKKPKVPSLPATQREDVIEEIIHKKPTAALSKFEDV\
KEHEEKETFVVLKKEIIDAPTKKEMVTAKHVIVPQKEEIIPSPTQEEVVSFKRKQTVRTS\
KKDAVPQKKEITYTQQTLEDKEEKILKRLEVTSTPDEEEIAHIQKKLYHTVRLVEKDVFP\
EKEDITMLETEEFVSQEIKLVSEPKPEKEKEIQGKKKVPPVSKKEEPLHHPKMDEKIVLK\
QKDVTLSHRKDEETVPQKKDPILALRKDEEIVTQKKDVTPPLIKEEESVPQNKDVTRPLR\
"""

print(s)
mlp = mlp.to(device).eval().requires_grad_(False)
emb = esmfold.infer_embedding(s)['s']

MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVISEPKPEKKEDIQLKKKEVVAVAKKEEVLKKEVVVPSKKDEEILPLKKEVPRPPKKEEDVMPQKKEVPRPPKKEEDIVPQMRDVSLPPKEEEKIVPKKKEVPRPPKKVEEILPPKKEVHRPPKKEEDIVPQIREVSLPPKKDEEIVCEKKEVAPAKEEPSKKPKVPSLPATQREDVIEEIIHKKPTAALSKFEDVKEHEEKETFVVLKKEIIDAPTKKEMVTAKHVIVPQKEEIIPSPTQEEVVSFKRKQTVRTSKKDAVPQKKEITYTQQTLEDKEEKILKRLEVTSTPDEEEIAHIQKKLYHTVRLVEKDVFPEKEDITMLETEEFVSQEIKLVSEPKPEKEKEIQGKKKVPPVSKKEEPLHHPKMDEKIVLKQKDVTLSHRKDEETVPQKKDPILALRKDEEIVTQKKDVTPPLIKEEESVPQNKDVTRPLR


In [76]:
sequence_logits = mlp(emb)

In [77]:
from plaid.utils import DecoderTokenizer
tokenizer = DecoderTokenizer()

from plaid.esmfold.misc import encode_sequence
true_aatype, _, _, _ = encode_sequence(s)
reimplemented_tokenizer_aatype = tokenizer.str_to_aatype_sequence(s)

(true_aatype == reimplemented_tokenizer_aatype).all()

tensor(True)

In [78]:
pred_aatype = sequence_logits.argmax(dim=-1).squeeze()
print(pred_aatype)

pred_aatype = pred_aatype.squeeze().cpu()
pred_s = tokenizer.aatype_to_str_sequence(pred_aatype)


tensor([12,  6,  6,  9, 16,  5,  9, 11, 11,  1, 10, 15,  5, 16, 19,  1, 10,  6,
         7, 11,  6,  3, 10, 10, 15, 11, 11,  3, 15,  9, 16,  2, 10, 11, 16,  6,
         6,  8, 19, 15, 19, 11, 11, 12, 19,  9, 15,  6, 14, 11, 14,  6, 11, 11,
         6,  3,  9,  5, 10, 11, 11, 11,  6, 19, 19,  0, 19,  0, 11, 11,  6,  6,
        19, 10, 11, 11,  6, 19, 19, 19, 14, 15, 11, 11,  3,  6,  6,  9, 10, 14,
        10, 11, 11,  6, 19, 14,  1, 14, 14, 11, 11,  6,  6,  3, 19, 12, 14,  5,
        11, 11,  6, 19, 14,  1, 14, 14, 11, 11,  6,  6,  3,  9, 19, 14,  5, 12,
         1,  3, 19, 15, 10, 14, 14, 11,  6,  6,  6, 11,  9, 19, 14, 11, 11, 11,
         6, 19, 14,  1, 14, 14, 11, 11, 19,  6,  6,  9, 10, 14, 14, 11, 11,  6,
        19,  8,  1, 14, 14, 11, 11,  6,  6,  3,  9, 19, 14,  5,  9,  1,  6, 19,
        15, 10, 14, 14, 11, 11,  3,  6,  6,  9, 19,  4,  6, 11, 11,  6, 19,  0,
        14,  0, 11,  6,  6, 14, 15, 11, 11, 14, 11, 19, 14, 15, 10, 14,  0, 16,
         5,  1,  6,  3, 19,  9,  6,  6, 

In [79]:
(pred_aatype == true_aatype).sum() / len(true_aatype)

tensor(1.)

In [80]:
print(s)
print(pred_s)
import numpy as np
(np.array(list(s)) == np.array(list(pred_s))).sum() / len(s)

MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVISEPKPEKKEDIQLKKKEVVAVAKKEEVLKKEVVVPSKKDEEILPLKKEVPRPPKKEEDVMPQKKEVPRPPKKEEDIVPQMRDVSLPPKEEEKIVPKKKEVPRPPKKVEEILPPKKEVHRPPKKEEDIVPQIREVSLPPKKDEEIVCEKKEVAPAKEEPSKKPKVPSLPATQREDVIEEIIHKKPTAALSKFEDVKEHEEKETFVVLKKEIIDAPTKKEMVTAKHVIVPQKEEIIPSPTQEEVVSFKRKQTVRTSKKDAVPQKKEITYTQQTLEDKEEKILKRLEVTSTPDEEEIAHIQKKLYHTVRLVEKDVFPEKEDITMLETEEFVSQEIKLVSEPKPEKEKEIQGKKKVPPVSKKEEPLHHPKMDEKIVLKQKDVTLSHRKDEETVPQKKDPILALRKDEEIVTQKKDVTPPLIKEEESVPQNKDVTRPLR
MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVISEPKPEKKEDIQLKKKEVVAVAKKEEVLKKEVVVPSKKDEEILPLKKEVPRPPKKEEDVMPQKKEVPRPPKKEEDIVPQMRDVSLPPKEEEKIVPKKKEVPRPPKKVEEILPPKKEVHRPPKKEEDIVPQIREVSLPPKKDEEIVCEKKEVAPAKEEPSKKPKVPSLPATQREDVIEEIIHKKPTAALSKFEDVKEHEEKETFVVLKKEIIDAPTKKEMVTAKHVIVPQKEEIIPSPTQEEVVSFKRKQTVRTSKKDAVPQKKEITYTQQTLEDKEEKILKRLEVTSTPDEEEIAHIQKKLYHTVRLVEKDVFPEKEDITMLETEEFVSQEIKLVSEPKPEKEKEIQGKKKVPPVSKKEEPLHHPKMDEKIVLKQKDVTLSHRKDEETVPQKKDPILALRKDEEIVTQKKDVTPPLIKEEESVPQNKDVTRPLR


1.0

In [82]:
# test that model loading classmethod works
model = FullyConnectedNetwork.from_pretrained()

In [84]:
sequence_logits = model(emb)
tokenizer.aatype_to_str_sequence(sequence_logits.argmax(dim=-1).squeeze())

'MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVISEPKPEKKEDIQLKKKEVVAVAKKEEVLKKEVVVPSKKDEEILPLKKEVPRPPKKEEDVMPQKKEVPRPPKKEEDIVPQMRDVSLPPKEEEKIVPKKKEVPRPPKKVEEILPPKKEVHRPPKKEEDIVPQIREVSLPPKKDEEIVCEKKEVAPAKEEPSKKPKVPSLPATQREDVIEEIIHKKPTAALSKFEDVKEHEEKETFVVLKKEIIDAPTKKEMVTAKHVIVPQKEEIIPSPTQEEVVSFKRKQTVRTSKKDAVPQKKEITYTQQTLEDKEEKILKRLEVTSTPDEEEIAHIQKKLYHTVRLVEKDVFPEKEDITMLETEEFVSQEIKLVSEPKPEKEKEIQGKKKVPPVSKKEEPLHHPKMDEKIVLKQKDVTLSHRKDEETVPQKKDPILALRKDEEIVTQKKDVTPPLIKEEESVPQNKDVTRPLR'